In [1]:
!pip install pandas
!pip install python-doctr
!pip install tf2onnx
!pip install tensorflow==2.10.0
!pip uninstall protobuf -y
!pip install protobuf==3.20.3

# other libraries accordingly


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 737.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.5/299.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 65.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=441e708df7b4d3234927f59a681f8af33f688ae0ae0341786006bb0489a388a2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

Found existing installation: protobuf 3.19.6
Uninstalling protobuf-3.19.6:
  Successfully uninstalled protobuf-3.19.6
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.23.1 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.10.0 which is incompatible.


In [2]:
!pip install protobuf==3.20.3

## Import

In [1]:
import pandas as pd
import re
from PIL import Image
import numpy as np
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import os
import requests

In [2]:


# Define unit symbols and their full names
unit_symbols = {
    'height': {'cm': 'centimetre', 'centimetre': 'centimetre', 'mm': 'millimetre', 'millimetre': 'millimetre', 'in': 'inch', 'inch': 'inch',
               'm': 'metre', 'metre': 'metre', 'ft': 'foot', 'foot': 'foot', 'yard': 'yard'},
    'width': {'cm': 'centimetre', 'centimetre': 'centimetre', 'mm': 'millimetre', 'millimetre': 'millimetre', 'in': 'inch', 'inch': 'inch',
              'm': 'metre', 'metre': 'metre', 'ft': 'foot', 'foot': 'foot', 'yard': 'yard'},
    'depth': {'cm': 'centimetre', 'centimetre': 'centimetre', 'mm': 'millimetre', 'millimetre': 'millimetre', 'in': 'inch', 'inch': 'inch',
              'm': 'metre', 'metre': 'metre', 'ft': 'foot', 'foot': 'foot', 'yard': 'yard'},
    'item_weight': {'g': 'gram', 'gm': 'gram', 'gms': 'gram', 'gram': 'gram', 'grams': 'gram',
                    'kg': 'kilogram', 'kilogram': 'kilogram', 'mg': 'milligram', 'milligram': 'milligram',
                    'µg': 'microgram', 'microgram': 'microgram', 'oz': 'ounce', 'ounce': 'ounce', 'ounces': 'ounce',
                    'lb': 'pound', 'lbs': 'pound', 'pound': 'pound', 'ton': 'ton', 'tons': 'ton'},
    'maximum_weight_recommendation': {'g': 'gram', 'gm': 'gram', 'gms': 'gram', 'gram': 'gram', 'grams': 'gram',
                                      'kg': 'kilogram', 'kilogram': 'kilogram', 'mg': 'milligram', 'milligram': 'milligram',
                                      'µg': 'microgram', 'microgram': 'microgram', 'oz': 'ounce', 'ounce': 'ounce', 'ounces': 'ounce',
                                      'lb': 'pound', 'lbs': 'pound', 'pound': 'pound', 'ton': 'ton', 'tons': 'ton'},
    'voltage': {'V': 'volt', 'v': 'volt', 'volt': 'volt', 'kV': 'kilovolt', 'kilovolt': 'kilovolt', 'mV': 'millivolt', 'millivolt': 'millivolt'},
    'wattage': {'W': 'watt', 'w': 'watt', 'watt': 'watt', 'kW': 'kilowatt', 'kw': 'kilowatt', 'kilowatt': 'kilowatt',
                'kWh': 'kilowatt hour', 'mAh': 'milliampere hour', 'hp': 'horsepower'},
    'item_volume': {'ml': 'millilitre', 'millilitre': 'millilitre', 'l': 'litre', 'L': 'litre', 'litre': 'litre',
                    'dl': 'decilitre', 'decilitre': 'decilitre', 'cl': 'centilitre', 'centilitre': 'centilitre',
                    'cup': 'cup', 'cups': 'cup', 'fl oz': 'fluid ounce', 'fluid ounce': 'fluid ounce', 'gallon': 'gallon',
                    'gal': 'gallon', 'imperial gallon': 'imperial gallon', 'pint': 'pint', 'pints': 'pint',
                    'quart': 'quart', 'qt': 'quart', 'cubic foot': 'cubic foot', 'ft³': 'cubic foot', 'ft3': 'cubic foot',
                    'cubic inch': 'cubic inch', 'in³': 'cubic inch', 'in3': 'cubic inch'}
}




In [3]:
# functions

def extract_text_from_image(image_url):
    response = requests.get(image_url)

    if response.status_code == 200:
        with open("image.jpg", "wb") as file:
            file.write(response.content)
    else:
        print("Failed to download the image.")
        # return " "
        exit()

    model = ocr_predictor(pretrained=True)
    doc = DocumentFile.from_images("image.jpg")
    result = model(doc)
    return ' '.join([word.value for block in result.pages[0].blocks for line in block.lines for word in line.words])



In [4]:

def find_value_with_symbol(text, entity_name):

    if entity_name == 'item_weight':
        net_weight_pattern = r'(?:net\s*(?:wt\.?|weight)|(?:total\s*)?weight|wt\.?|w\.)\s*[:()]?\s*(\d+(?:[.,]\d+)?)\s*([a-zA-Z]+)'
        match = re.search(net_weight_pattern, text, re.IGNORECASE)
        if match:
            value, unit = match.groups()
            value = value.replace(',', '.')
            formatted_value = f"{float(value):.1f}" if '.' not in value else value

            ocr_corrections = {'c': 'g'}
            unit = ocr_corrections.get(unit.lower(), unit)

            full_unit_name = next((full for abbr, full in unit_symbols['item_weight'].items()
                                   if abbr.lower() == unit.lower()), unit)

            print(f"Found net weight: {formatted_value} {full_unit_name}")
            return f"{formatted_value} {full_unit_name}"

    elif entity_name == 'width':
        width_pattern = r'(\d+(?:[.,]\d+)?)\s*([a-zA-Z]+)'
        matches = re.findall(width_pattern, text, re.IGNORECASE)
        for value, unit in matches:
            if unit.lower() in [u.lower() for u in unit_symbols.get('width', {}).keys()]:
                value = value.replace(',', '.')
                formatted_value = f"{float(value):.1f}" if '.' not in value else value
                full_unit_name = next((full for abbr, full in unit_symbols['width'].items()
                                       if abbr.lower() == unit.lower()), unit)
                print(f"Found width: {formatted_value} {full_unit_name}")
                return f"{formatted_value} {full_unit_name}"

    elif entity_name in ['depth', 'height']:
        dimension_pattern = r'(\d+(?:[.,]\d+)?)\s*([a-zA-Z]+)'
        matches = re.findall(dimension_pattern, text, re.IGNORECASE)
        valid_matches = []
        for value, unit in matches:
            if unit.lower() in [u.lower() for u in unit_symbols.get(entity_name, {}).keys()]:
                valid_matches.append((value, unit))

        if valid_matches:
            value, unit = valid_matches[-1]  # Select the last valid match
            value = value.replace(',', '.')
            formatted_value = f"{float(value):.1f}" if '.' not in value else value
            full_unit_name = next((full for abbr, full in unit_symbols[entity_name].items()
                                   if abbr.lower() == unit.lower()), unit)
            print(f"Found {entity_name}: {formatted_value} {full_unit_name}")
            return f"{formatted_value} {full_unit_name}"

    # If no specific condition is met, use the original logic
    symbols = unit_symbols.get(entity_name, {})
    for symbol, unit_name in symbols.items():
        pattern = rf'(\d+(?:[.,]\d+)?)\s*{re.escape(symbol)}'
        match = re.search(pattern, text)
        if match:
            value = match.group(1).replace(',', '.')
            formatted_value = f"{float(value):.1f}" if '.' not in value else value
            print(f"Found {entity_name}: {formatted_value} {unit_name}")
            return f"{formatted_value} {unit_name}"

    print(f"No {entity_name} found")
    return ''

In [ ]:
def process_row(row):
    extracted_text = extract_text_from_image(row['image_link']).lower()
    print(extracted_text) ####
    return find_value_with_symbol(extracted_text, row['entity_name'])

def main():
    try:
        chunk_size = 10000  # Set the chunk size to 10,000 rows
        chunk_no = 1  # Track the chunk number

        # Iterate over the CSV in chunks
        for chunk in pd.read_csv('test.csv', chunksize=chunk_size):

            # Apply the processing function to each row in the chunk
            chunk['extracted_value'] = chunk.apply(process_row, axis=1)

            # Save the chunk with predictions
            output_filename = f'test_output_{chunk_no}.csv'
            chunk.to_csv(output_filename, index=False)

            # required prediction output format
            po = chunk[['extracted_value']].copy()  # Extract the 'extracted_value' column
            po['index'] = chunk.index  # Add the index as a separate column
            po.rename(columns={'extracted_value': 'prediction'}, inplace=True)  # Rename the column to 'prediction'

            # Reorder the columns to have 'index' first
            po = po[['index', 'prediction']]
            po.to_csv(f"prediction_output_{chunk_no}.csv", index=False)

            print(f"Chunk {chunk_no} processed and saved as {output_filename}")
            chunk_no += 1

        print("Processing complete. All chunks saved.")

    except Exception as e:
        print(f"An error occurred: {str(e)}")


if __name__ == "__main__":
    main()


premium materials r dreomxong tire grade solid i rubber material i i i --- sturdy steel : - - : / withstand weight up to 500 pounds, preventing the gate from sagging.
Found maximum_weight_recommendation: 500.0 pound
zf stand bag zero friction. soft and durable 72 zipper pockets for handle versatile storage weighs under 5.5 ibs. 6-way full length easy-carry bottom of bag dual strap dividers - comfortable hip pad essential glove and towel holder
No item_weight found
premiun feature - day & night use of up to 984ft / 300m 10x optical zoom ofo le 7ir levels & 4x digital zoom photography & video water resistant ip 56
No item_weight found
a : c - 8 l 2 c r a i ly e 117  i a landshark premium - - - quality - - laaex - fl.oz. island style lager bottles 12-12
Found item_volume: 8.0 litre
xx  - a 9 & a x - sn s 3 a a a s alto ofession a 700-watt alto tx312 12-inch 2-way p rofessional power loudspeaker
No item_weight found
spicebomb wwe bin viktor (& rolf eau de toilette pour homme vaporisateur n

## Evaluation

In [ ]:
df = pd.read_csv('complete_test_output.csv')
df

,image_link,group_id,entity_name,entity_value,extracted_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,2.0 litre
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,0.709 gram


In [ ]:

# Function to split and normalize value and unit
def normalize(value):
    if isinstance(value, str):
        # Use regex to split the numeric part and the unit part
        match = re.match(r'(\d+(\.\d+)?)\s*(\w+)', value.strip().lower())
        if match:
            number = float(match.group(1))  # Convert numeric part to float
            unit = match.group(3)  # Extract the unit part
            return (number, unit)
    return value

# Initialize counters
true_positives = 0
false_positives = 0
false_negatives = 0
true_negatives = 0

# Define the logic to classify predictions
for _, row in df.iterrows():
    gt = normalize(row['entity_value'])
    out = normalize(row['extracted_value'])

    if isinstance(out, tuple) and isinstance(gt, tuple):
        # If both GT and OUT are tuples (number, unit), compare both
        if out == gt:
            true_positives += 1
        else:
            false_positives += 1
    elif out != "" and gt != "":
        # If both are non-empty strings but not tuples, directly compare
        if out == gt:
            true_positives += 1
        else:
            false_positives += 1
    elif out != "" and gt == "":
        false_positives += 1
    elif out == "" and gt != "":
        false_negatives += 1
    elif out == "" and gt == "":
        true_negatives += 1

# Calculate Precision and Recall
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Calculate F1 Score
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")
print(f"True Negatives: {true_negatives}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")


True Positives: 2
False Positives: 1
False Negatives: 0
True Negatives: 0
Precision: 0.67
Recall: 1.00
F1 Score: 0.80
